## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2024-04-16 01:32:35.578424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 01:32:36.427796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1


In [2]:
# untar
!ls
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

CNN_pruning_students_.ipynb	  basic2.h5	    train_labels.pkl
CNN_pruning_students_glenn.ipynb  dataset.tar.gz    val_images.pkl
basic.h5			  train_images.pkl  val_labels.pkl
train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [3]:
# Define the neural network architecture (don't change this)
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

2024-04-16 01:32:40.944697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22295 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:e1:00.0, compute capability: 8.6


In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 11, 11, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [5]:
# you can use the default hyper-parameters for training,
# val accuracy ~72% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


2024-04-16 01:32:51.947789: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-16 01:32:52.144262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2024-04-16 01:32:53.261075: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-16 01:32:53.273497: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9f855f28b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-16 01:32:53.273539: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-16 01:32:53.281135: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_uti

703/703 [==============================] - 8s 6ms/step - loss: 1.5088 - accuracy: 0.3160 - val_loss: 1.3464 - val_accuracy: 0.4246
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3447 - accuracy: 0.4205 - val_loss: 1.2630 - val_accuracy: 0.4594
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2848 - accuracy: 0.4576 - val_loss: 1.2259 - val_accuracy: 0.4911
Epoch 4/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2352 - accuracy: 0.4855 - val_loss: 1.1712 - val_accuracy: 0.5081
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2041 - accuracy: 0.5022 - val_loss: 1.1381 - val_accuracy: 0.5366
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1695 - accuracy: 0.5223 - val_loss: 1.1121 - val_accuracy: 0.5406
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1413 - accuracy: 0.5370 - val_loss: 1.1003 - val_accuracy: 0.5501
Epoch 8/50
703/70

In [7]:
results = model.evaluate(val_images, val_labels, batch_size=128)

 1/20 [>.............................] - ETA: 0s - loss: 1.0630 - accuracy: 0.5938

20/20 [==============================] - 0s 3ms/step - loss: 0.7658 - accuracy: 0.6958


In [ ]:
# # perform pruning here

# # get the weights
# weights = model.get_weights()

# # you can use set_weights() to set some weights to zero, e.g.,

# weights[7][:10]=0
# model.set_weights(weights)

### Next, we use the method introduced in the paper [Data-Driven Sparse Structure Selection for Deep Neural Networks](https://arxiv.org/pdf/1707.01213.pdf). We adjust the original method to this case.

#### First implement sparse scaling layer (supporting weights).

In [9]:
from tensorflow.keras.layers import Layer
import tensorflow as tf

class SparseScaling(Layer):
    def __init__(self, **kwargs):
        super(SparseScaling, self).__init__(**kwargs)
        self.gamma = 0.01

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale', 
                                     shape=(input_shape[-1],),
                                     initializer='ones',
                                     regularizer=tf.keras.regularizers.l1(self.gamma),
                                     trainable=True)
        super(SparseScaling, self).build(input_shape)

    def call(self, inputs):
        return inputs * self.scale

#### Next, insert this layer after each existing layer.
#####
 We found that since the sparse layer already acts as a penalty term added to the original weights, the l2 regularizer can be removed here, and experiments have proven that this will yield better results.

In [10]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [11]:
# take a look at the structure
weights = model.get_weights()
for i in range(len(weights)):
    print(i, weights[i].shape)

0 (3, 3, 3, 32)
1 (32,)
2 (32,)
3 (3, 3, 32, 32)
4 (32,)
5 (32,)
6 (3, 3, 32, 64)
7 (64,)
8 (64,)
9 (3, 3, 64, 64)
10 (64,)
11 (64,)
12 (1024, 512)
13 (512,)
14 (512,)
15 (512, 5)
16 (5,)


In [12]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 25, 25, 32)        896       
                                                                 
 activation_7 (Activation)   (None, 25, 25, 32)        0         
                                                                 
 sparse_scaling (SparseScal  (None, 25, 25, 32)        32        
 ing)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_8 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 sparse_scaling_1 (SparseSc  (None, 23, 23, 32)        32        
 aling)                                               

In [13]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50,
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


2024-04-16 01:39:22.300390: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/dropout_3/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


703/703 [==============================] - 7s 6ms/step - loss: 6.1113 - accuracy: 0.3566 - val_loss: 3.6930 - val_accuracy: 0.4333
Epoch 2/50
703/703 [==============================] - 4s 6ms/step - loss: 2.2570 - accuracy: 0.4047 - val_loss: 1.7222 - val_accuracy: 0.3925
Epoch 3/50
703/703 [==============================] - 4s 6ms/step - loss: 1.6825 - accuracy: 0.4026 - val_loss: 1.5998 - val_accuracy: 0.4250
Epoch 4/50
703/703 [==============================] - 4s 6ms/step - loss: 1.6006 - accuracy: 0.4160 - val_loss: 1.5276 - val_accuracy: 0.4242
Epoch 5/50
703/703 [==============================] - 4s 6ms/step - loss: 1.5523 - accuracy: 0.4288 - val_loss: 1.5025 - val_accuracy: 0.4218
Epoch 6/50
703/703 [==============================] - 5s 7ms/step - loss: 1.5134 - accuracy: 0.4386 - val_loss: 1.4657 - val_accuracy: 0.4550
Epoch 7/50
703/703 [==============================] - 5s 7ms/step - loss: 1.4954 - accuracy: 0.4367 - val_loss: 1.4847 - val_accuracy: 0.4115
Epoch 8/50
703/70

In [671]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 0.8912 - accuracy: 0.7232


### Next mask out the structure (kernels, neurons, ...) whose corresponding scaling factor is below a threshold.

In [14]:
def prune_weights(model):
    for i, layer in enumerate(model.layers):
        if isinstance(layer, SparseScaling):
            scale_factors = np.abs(layer.scale.numpy())
            if i == 2: # first conv layer
                previous_layer = model.layers[i-2]
                weights, biases = previous_layer.get_weights()
                new_weights = weights * (scale_factors >= 0.1)
                previous_layer.set_weights([new_weights, biases])
            if i == 5: # second conv layer
                previous_layer = model.layers[i-2]
                weights, biases = previous_layer.get_weights()
                new_weights = weights * (scale_factors >= 0.1)
                previous_layer.set_weights([new_weights, biases])
            if i == 10: # third conv layer
                previous_layer = model.layers[i-2]
                weights, biases = previous_layer.get_weights()
                new_weights = weights * (scale_factors >= 0.155)
                previous_layer.set_weights([new_weights, biases])
            if i == 13: # forth conv layer
                previous_layer = model.layers[i-2]
                weights, biases = previous_layer.get_weights()
                new_weights = weights * (scale_factors >= 0.06)
                previous_layer.set_weights([new_weights, biases])
            if i == 19: # first dense layer
                previous_layer = model.layers[i-2]
                weights, biases = previous_layer.get_weights()
                new_weights = weights * (scale_factors >= 0.035)
                previous_layer.set_weights([new_weights, biases])

prune_weights(model)

### Remove the supporting weights (scaling factors).

In [17]:
def transfer_weights(original_model, new_model):
    original_layers = [layer for layer in original_model.layers if not isinstance(layer, SparseScaling)]
    new_layers = new_model.layers
    assert len(original_layers) == len(new_layers), "Layer count must match between the models."
    for orig_layer, new_layer in zip(original_layers, new_layers):
        if isinstance(orig_layer, type(new_layer)):
            new_layer.set_weights(orig_layer.get_weights())


new_model = models.Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(25,25,3)),
    Activation('relu'),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512),
    Activation('relu'),
    Dropout(0.5),
    Dense(5),
    Activation('softmax')
])

transfer_weights(model, new_model)

#### Retrain the model with deprecated structures masked out.

In [18]:
from tensorflow.keras.callbacks import Callback

class PruneWeights(Callback):
    def __init__(self, masks):
        super(PruneWeights, self).__init__()
        self.masks = masks

    # def on_epoch_end(self, epoch, logs=None):
    #     for i, layer in enumerate(self.model.layers):
    #         if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
    #             weights, biases = layer.get_weights()
    #             weights *= self.masks[i]
    #             layer.set_weights([weights, biases])

    def on_train_batch_end(self, batch, logs=None):
        for i, layer in enumerate(self.model.layers):
            if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
                weights, biases = layer.get_weights()
                weights *= self.masks[i]
                layer.set_weights([weights, biases])
    

def generate_pruning_mask(model):
    mask = []
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
            weights, biases = layer.get_weights()
            layer_mask = (weights != 0).astype(float)
            mask.append(layer_mask)
        else:
            mask.append(None)
    return mask

In [21]:
pruning_mask = generate_pruning_mask(new_model)
prune_callback = PruneWeights(masks=pruning_mask)
# print(pruning_mask[0])

In [20]:
# save model while training
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'my_model_weights_best.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1)

In [ ]:
new_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
new_model.fit(train_images, train_labels, batch_size=32, epochs=500, validation_data=(val_images, val_labels), callbacks=[prune_callback, model_checkpoint_callback])

In [ ]:
new_model.load_weights('my_model_weights_best.h5')

results = new_model.evaluate(val_images, val_labels, batch_size=128)

def calculate_zero_weights_ratio(model):
    total_weights = 0
    zero_weights = 0
    for layer in model.layers:
        if hasattr(layer, 'get_weights') and len(layer.get_weights()) > 0:
            weights = layer.get_weights()[0]
            total_weights += np.size(weights)
            zero_weights += np.count_nonzero(weights == 0)
    if total_weights == 0:
        return 0
    else:
        return zero_weights / total_weights

calculate_zero_weights_ratio(new_model)

### We also combine this method with L1 norm.

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(25,25,3), kernel_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l1(0.01)))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512), kernel_regularizer=regularizers.l1(0.01))
model.add(Activation('relu'))
model.add(SparseScaling())
model.add(Dropout(0.5))
model.add(Dense(5), kernel_regularizer=regularizers.l1(0.01))
model.add(Activation('softmax'))

In [ ]:
model.load_weights('my_model_weights_best.h5')

pruning_mask = generate_pruning_mask(model)
prune_callback = PruneWeights(masks=pruning_mask)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
model.fit(train_images, train_labels, batch_size=32, epochs=200, validation_data=(val_images, val_labels), callbacks=[prune_callback])

In [ ]:
weights = model.get_weights()
for i in range(len(weights)):
    layer_weights = model.get_weights()[i]
    modified_weights = np.where(np.abs(layer_weights) < 0.1, 0, layer_weights)
    weights[i] = modified_weights
    model.set_weights(weights)

In [77]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
new_model.save_weights("my_model_weights.h5")

In [ ]:
# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")